In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets

In [2]:
data_path_sniim = os.path.join("..","..","data","clean_data","precios_sniim.parquet")
data_path_siap = os.path.join("..","..","data","clean_data","siap.parquet.zip")
data_path_siap_wide = os.path.join("..","..","data","clean_data","siap_wide.parquet.zip")

In [3]:
df_precios_sniim = pd.read_parquet(data_path_sniim)
df_cultivos_siap = pd.read_parquet(data_path_siap)
df_cultivos_siap_wide = pd.read_parquet(data_path_siap_wide)

In [4]:
df_precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
36399,2021-12-23,Piña Grande,Pieza,Quintana Roo,Quintana Roo,"Mercado de Chetumal, Quintana Roo",35.00,35.00,35.00
2009748,2021-08-10,Mango Ataulfo,Caja de 30 kg.,Chiapas,Durango,Centro de Distribución y Abasto de Gómez Palacio,20.00,21.67,21.67
1582360,2023-04-05,Uva Globo,Caja de 8 kg.,Sonora,San Luis Potosí,Centro de Abasto de San Luis Potosí,68.75,70.00,68.75
235097,2021-10-26,Cebolla De Rabo,Kilogramo,Jalisco,Colima,Centros de distribución de Colima,16.00,20.00,20.00
1914127,2023-08-25,Pera D´Anjou #100,Caja de 20 kg.,Importación,Coahuila,"Central de Abasto de La Laguna, Torreón",40.00,48.00,48.00
789586,2020-03-19,Naranja Valencia Grande,Caja de 20 kg.,Yucatán,Yucatán,"Mercado ""Casa del Pueblo""",9.00,9.00,9.00
751768,2020-11-24,Ajo Morado,Caja de 10 kg.,Guanajuato,México,Central de Abasto de Toluca,75.00,80.00,80.00
1121433,2023-08-17,Calabacita Italiana,Caja de 16 kg.,Veracruz,Veracruz,Central de Abasto de Jalapa,13.13,13.44,13.44
242559,2020-02-20,Col Grande,Kilogramo,Guanajuato,Guanajuato,Central de Abasto de León,15.00,16.00,15.00
1743798,2021-03-05,Chile Ancho,Kilogramo,Distrito Federal,Hidalgo,Central de Abasto de Pachuca,90.00,95.00,90.00


In [5]:
df_cultivos_siap_wide.sample(10)

,Entidad,Municipio,Sembrada (ha),Cosechada (ha),Siniestrada (ha),Producción,Rendimiento (udm/ha),Cultivo,fecha
46,Guanajuato,Tarimoro,4.00,4.00,0.0,89.00,22.25,pepino,2021-12-01
168,Puebla,Santa Isabel Cholula,6.00,0.00,0.0,0.00,0.00,cebolla,2020-01-01
52,Jalisco,Encarnación de Díaz,2.00,2.00,0.0,105.00,52.50,nopalitos,2022-12-01
265,Veracruz,Tlalixcoyan,137.32,0.00,0.0,0.00,0.00,chile verde,2022-01-01
1398,Sonora,Alamos,250.00,0.00,0.0,0.00,0.00,frijol,2021-09-01
542,Oaxaca,Villa de Etla,1.50,1.50,0.0,57.30,38.20,avena forrajera en verde,2022-07-01
363,Veracruz,Teocelo,59.00,0.00,0.0,0.00,0.00,mango,2023-01-01
85,Sonora,Hermosillo,4.00,4.00,0.0,144.00,36.00,coliflor,2020-05-01
247,Michoacán,Huandacareo,66.88,66.88,0.0,342.89,5.13,sorgo grano,2021-12-01
71,Michoacán,Copándaro,19.78,10.00,0.0,220.00,22.00,lechuga,2020-08-01


In [6]:
df_precios_sniim["año"] = df_precios_sniim["fecha"].dt.year
df_precios_sniim["mes"] = df_precios_sniim["fecha"].dt.month

In [56]:
def graficar_precios(cultivo):
    subset_df = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
    
    subset_df = subset_df.resample('D', on='fecha').mean(numeric_only=True)
    subset_df["ewm"] = subset_df["precio_frec"].ewm(span=14).mean()
    #subset_df["año"] = subset_df.index.year
    
    fig,ax = plt.subplots()
    
    sns.lineplot(x="fecha",
                 y="ewm",
                 hue="año",
                 data=subset_df,
                 errorbar=None,
                 ax=ax,
                 palette="tab10")
    
    sns.scatterplot(x="fecha",
                    y="precio_frec",
                    hue="año",
                    data=subset_df,
                    ax=ax,
                    palette="tab10",
                    s=25,
                    alpha=0.5)
    
    ax.title.set_text(f"Historial de precios para {cultivo} (promedio diario)")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Precio por kg, MXN")
    ax.grid(axis='y')
    ax.tick_params(axis='x',rotation=90);
    ax.legend(title='Año', labels=["2020","2021","2022","2023"])
    
    fig.set_size_inches(16,9)
    
    plt.show()
    
    return

In [57]:
#graficar_precios("Papaya Maradol")

In [58]:
lista_productos = sorted(list(df_precios_sniim["producto"].unique()))
lista_destinos = sorted(list(df_precios_sniim["destino"].unique()))
lista_anios = [2020,2021,2022,2023]
#lista_productos[:5]

In [59]:
dd_productos = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

widgets.interact(graficar_precios,cultivo=dd_productos)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.graficar_precios(cultivo)>

In [64]:
def grafica_anio(cultivo,anio):
    subset_df = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
    subset_df = subset_df[subset_df["año"] == anio]
    
    try:
        subset_df = subset_df.resample('D', on='fecha').mean(numeric_only=True)
        subset_df["ewm"] = subset_df["precio_frec"].ewm(span=7).mean()

        fig,ax = plt.subplots()

        sns.lineplot(x="fecha",
                     y="ewm",
                     data=subset_df,
                     errorbar=None,
                     ax=ax)

        sns.scatterplot(x="fecha",
                        y="precio_frec",
                        data=subset_df,
                        ax=ax,
                        s=25,
                        alpha=0.5)

        ax.title.set_text(f"Historial de precios para {cultivo}, año: {anio} (promedio diario)")
        ax.set_xlabel("Fecha")
        ax.set_ylabel("Precio por kg, MXN")
        ax.grid(axis='y')
        ax.tick_params(axis='x',rotation=90);

        fig.set_size_inches(16,9)
        
        subset_df2 = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
        subset_df2 = subset_df2[subset_df2["año"] == anio]
        
        fecha_max = subset_df2[subset_df2["precio_frec"] == subset_df2["precio_frec"].max()]["fecha"].iloc[0]
        central_max = subset_df2[subset_df2["precio_frec"] == subset_df2["precio_frec"].max()]["central"].iloc[0]
        precio_max = subset_df2["precio_frec"].max()
        
        print(f"Precio maximo: ${precio_max:.2f} el {fecha_max.strftime('%Y-%d-%m')} en {central_max}")
        
        fecha_min = subset_df2[subset_df2["precio_frec"] == subset_df2["precio_frec"].min()]["fecha"].iloc[0]
        central_min = subset_df2[subset_df2["precio_frec"] == subset_df2["precio_frec"].min()]["central"].iloc[0]
        precio_min = subset_df2["precio_frec"].min()
        
        print(f"Precio minimo: ${precio_min:.2f} el {fecha_min.strftime('%Y-%d-%m')} en {central_min}")

        plt.show()
    
    except:
        print("No hay datos para esa combinacion")
    
    return

In [65]:
dd_productos = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

dd_anios = widgets.Dropdown(options=lista_anios,
                      value=lista_anios[0],
                      description="Anio: ")

widgets.interact(grafica_anio,cultivo=dd_productos,anio=dd_anios)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.grafica_anio(cultivo, anio)>

In [14]:
df_copy = df_precios_sniim.copy()

In [66]:
df_filter = df_copy[(df_copy["producto"] == "Ajo Blanco") & (df_copy["año"] == 2022)]

In [67]:
df_grouped = df_filter.groupby(["central","mes"])["precio_frec"].mean().to_frame()
df_grouped.columns = ["Precio promedio"]

In [68]:
df_grouped.unstack()

Precio promedio              \
mes                                                           1           2    
central                                                                        
Central de Abasto "Francisco Villa"                    85.416667         NaN   
Central de Abasto INDIA, Tijuana                      154.000000  129.263158   
Central de Abasto de Culiacán                          75.523810   65.052632   
Central de Abasto de Jalapa                            83.333333   78.315789   
Central de Abasto de León                              95.000000   95.315789   
Central de Abasto de Minatitlán                        79.047619   70.000000   
Central de Abasto de Tuxtla Gutiérrez                  65.952381   71.842105   
Central de Abasto de Villahermosa                      60.000000   60.000000   
Centro Comercial Agropecuario de Aguascalientes        50.000000   41.578947   
Centros de distribución de Colima                      85.750000   85.000000   
Mercado Felipe Ángeles de Guadalajara                  54.000000   55.578947   
Mercado Malibrán                                       70.000000   72.631579   
Mercado de Abasto de Guadalajara                       43.000000   50.578947   
Mercado de Chetumal, Quintana Roo                     120.000000  120.000000   
Mercado de abasto 'Adolfo López Mateos' de Tepic      116.523810  114.157895   
Nayarabastos de Tepic                                 118.850000  114.526316   
Unión de Comerciantes de La Paz                        88.187500  105.263158   

                                                                          \
mes                                                       3           4    
central                                                                    
Central de Abasto "Francisco Villa"                      NaN         NaN   
Central de Abasto INDIA, Tijuana                  123.090909  144.631579   
Central de Abasto de Culiacán                      64.136364   63.684211   
Central de Abasto de Jalapa                        62.136364   50.684211   
Central de Abasto de León                          88.409091   84.210526   
Central de Abasto de Minatitlán                    74.545455   65.789474   
Central de Abasto de Tuxtla Gutiérrez              69.090909   64.473684   
Central de Abasto de Villahermosa                  60.000000   68.684211   
Centro Comercial Agropecuario de Aguascalientes    38.545455   46.315789   
Centros de distribución de Colima                  85.000000   87.000000   
Mercado Felipe Ángeles de Guadalajara              56.500000   56.500000   
Mercado Malibrán                                   71.818182   65.789474   
Mercado de Abasto de Guadalajara                   55.000000   55.000000   
Mercado de Chetumal, Quintana Roo                 120.000000  120.000000   
Mercado de abasto 'Adolfo López Mateos' de Tepic  118.136364  119.894737   
Nayarabastos de Tepic                             117.045455  119.368421   
Unión de Comerciantes de La Paz                   100.000000   98.947368   

                                                                          \
mes                                                       5           6    
central                                                                    
Central de Abasto "Francisco Villa"                      NaN         NaN   
Central de Abasto INDIA, Tijuana                  146.428571  123.181818   
Central de Abasto de Culiacán                      63.428571   64.681818   
Central de Abasto de Jalapa                        48.190476   58.545455   
Central de Abasto de León                          78.809524   85.454545   
Central de Abasto de Minatitlán                    68.095238   70.000000   
Central de Abasto de Tuxtla Gutiérrez              68.809524   67.727273   
Central de Abasto de Villahermosa                  70.000000   70.000000   
Centro Comercial Agropecuario de Aguascalientes    48.714286   47.000000   
Centros de dist